In [1]:
%load_ext jupyter_black

# nexrad

In [5]:
import re
from datetime import datetime
from typing import NewType, Iterable
# 
import pandas as pd
import numpy as np
# connecting
import boto3
import botocore
from botocore.client import Config

import matplotlib.pyplot as plt
from metpy.io import Level2File
from metpy.plots import add_timestamp, ctables
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [13]:
s3 = boto3.resource(
    "s3",
    config=Config(signature_version=botocore.UNSIGNED, user_agent_extra="Resource"),
)
bucket = s3.Bucket("noaa-nexrad-level2")


class NexradSummary:
    def __init__(self, dsum: pd.Series) -> None:
        self.__date_range_summary = dsum

    def __repr__(self) -> str:
        return self.__date_range_summary.__repr__()

    def itersum(self):
        yield from self.__date_range_summary.items()

    def _generator(self, key: bytes, sweep: int):
        for vt, obj in self.itersum():
            f = Level2File(obj.get()["Body"])

    def reflectivity(self, sweep: int = 0):
        return tuple(self._generator(b"REF", sweep))

        #     # return f
        #     sweep = 0
            # # First item in ray is header, which has azimuth angle
            # az = np.array([ray[0].az_angle for ray in f.sweeps[sweep]])

            # ref_hdr = f.sweeps[sweep][0][4][b"REF"][0]
            # ref_range = (
            #     np.arange(ref_hdr.num_gates) * ref_hdr.gate_width + ref_hdr.first_gate
            # )
            # self.__ref = np.array([ray[4][b"REF"][1] for ray in f.sweeps[sweep]])

            # rho_hdr = f.sweeps[sweep][0][4][b"RHO"][0]
            # rho_range = (
            #     np.arange(rho_hdr.num_gates + 1) - 0.5
            # ) * rho_hdr.gate_width + rho_hdr.first_gate
            # rho = np.array([ray[4][b"RHO"][1] for ray in f.sweeps[sweep]])

            # phi_hdr = f.sweeps[sweep][0][4][b"PHI"][0]
            # phi_range = (
            #     np.arange(phi_hdr.num_gates + 1) - 0.5
            # ) * phi_hdr.gate_width + phi_hdr.first_gate
            # phi = np.array([ray[4][b"PHI"][1] for ray in f.sweeps[sweep]])

        #     zdr_hdr = f.sweeps[sweep][0][4][b"ZDR"][0]
            # zdr_range = (
            #     np.arange(zdr_hdr.num_gates + 1) - 0.5
            # ) * zdr_hdr.gate_width + zdr_hdr.first_gate
        #     zdr = np.array([ray[4][b"ZDR"][1] for ray in f.sweeps[sweep]])


def nexrad(date: datetime, station_id: str):
    prefix = date.strftime(f"%Y/%m/%d/{station_id}")

    def generate():
        for obj in bucket.objects.filter(Prefix=prefix):
            key = obj.key.split("/")[-1]
            if not key.endswith("_V06"):
                continue
            d = datetime.strptime(key, f"{station_id}%Y%m%d_%H%M%S_V06")

            return d, obj
            # if True:
            #     yield d, obj

    # dsum = pd.DataFrame(generate(), columns=["validTime", "summary"]).set_index("validTime")["summary"]

    return generate()


d, lvl2 = nexrad(datetime(2022, 7, 20, 0, 19, 5), "KMLB")
lvl2
# lvl2.reflectivity()

s3.ObjectSummary(bucket_name='noaa-nexrad-level2', key='2022/07/20/KMLB/KMLB20220720_000314_V06')

In [87]:
sweep = 0
f = Level2File(lvl2.get()["Body"])
# az = np.array([ray[0].az_angle for ray in f.sweeps[sweep]])

In [222]:
from typing import NewType, NamedTuple
import xarray as xr

DataBlockHdr = NewType("DataBlockHdr", NamedTuple)
# DataBlockHdr(type=b'D', name=b'REF', reserved=0, num_gates=1832, first_gate=2.125, gate_width=0.25, tover=5.0, snr_thresh=0.0, recombined=None, data_size=8, scale=2.0, offset=66.0)
ref_hdr: DataBlockHdr = f.sweeps[sweep][0][4][b"REF"][0]
ref_range = (
    # num_gates=1832[] *  gate_width=0.25 + first_gate=2.125
    np.arange(ref_hdr.num_gates) * ref_hdr.gate_width
    + ref_hdr.first_gate
)
ref_hdr
# az
def iterkeys():
    yield from ((bkey.decode("utf-8"), bkey) for bkey in (b"REF", b"ZDR", b"PHI", b"RHO", b"CFP"))


from typing import NamedTuple


class GateInfo(NamedTuple):
    first_gate: float
    gate_width: float
    num_gates: float


def arange_gates(gate: GateInfo):
    return np.arange(gate.num_gates, dtype=np.float32) * gate.gate_width + gate.first_gate


class Sweep:
    def __init__(self, sweeps: list) -> None:
        for s in sweeps:
            for ray in s:
                self.value = ray[4][b"REF"][1]

        # pass


class RadarInterface:
    def __init__(
        self,
        level2: Level2File,
    ) -> None:
        self.__level2 = level2

    def __getitem__(self, ix):
        # sweeps = self.__level2.sweeps[ix]

        sweep_slice = self.__level2.sweeps[ix]
        if not isinstance(ix, slice):
            sweep_slice = [sweep_slice]
        return Sweep(sweep_slice)
        # def generate():
        #     if isinstance(ix, slice):
        #         for sweep1 in sweep_slice:
        #             yield [ray[4] for ray in sweep_slice]
        #     else:
        #         yield [[ray[4] for ray in sweep_slice]]

        # return np.array(tuple(generate()))
        ...


class Nexgen:
    def __init__(self, lvl2: Level2File, sweep=0):
        # First item in ray is header, which has azimuth angle
        swept = lvl2.sweeps[sweep]
        az = np.array([ray[0].az_angle for ray in swept])
        diff = np.diff(az)
        diff[diff > 180] -= 360.0
        diff[diff < -180] += 360.0
        avg_spacing = diff.mean()
        az = (az[:-1] + az[1:]) / 2
        az = np.concatenate(([az[0] - avg_spacing], az, [az[-1] + avg_spacing]))
        sweep_range = np.arange(len(swept))

        level_4 = tuple(ray[4] for ray in lvl2.sweeps[sweep])

        ref_range, zdr_range, phi_range, rho_range, cfp_range = (
            arange_gates(
                lvl2.sweeps[sweep][0][4][bKey][0],
            )
            for bKey in (b"REF", b"ZDR", b"PHI", b"RHO", b"CFP")
        )

        # xlocs = var_range * np.sin(np.deg2rad(az[:, np.newaxis]))
        # ylocs = var_range * np.cos(np.deg2rad(az[:, np.newaxis]))
        # x, y = zdr_range * np.sin(np.deg2rad(az[:, np.newaxis])), zdr_range * np.cos(np.deg2rad(az[:, np.newaxis]))
        azrads = np.deg2rad(az[:, np.newaxis])
        azsin, az_cos = (
            np.sin(azrads),
            np.cos(azrads),
        )
        # a = np.array(np.meshgrid(x, y), dtype=np.float32)
        # print(a)
        coords = {
            "sweeps": (["Z"], sweep_range),
            # 
            "ref_range": (["R0"], ref_range),
            "xloc_0": (("i0", "j0"), ref_range * azsin),
            "yloc_0": (("i0", "j0"), ref_range * az_cos),
            # 
            "zdr_range": (["R1"], zdr_range),
            "xloc_1": (("i1", "j1"), zdr_range * azsin),
            "yloc_1": (("i1", "j1"), zdr_range * az_cos),
        }


        data_vars = {
            "reflectivity": (["z", "R0"], [ray[b"REF"][1] for ray in level_4]),
            "cfp": (["z", "R0"], [ray[b"CFP"][1] for ray in level_4]),
            "differential_reflectivity": (["z", "R1"], [ray[b"ZDR"][1] for ray in level_4]),
            "differential_phase_shift": (["z", "R1"], [ray[b"PHI"][1] for ray in level_4]),
            "correlation_coefficient": (["z", "R1"], [ray[b"RHO"][1] for ray in level_4]),
        }

        # return data_vars
        ds = xr.Dataset(
            data_vars=data_vars,
            coords=coords,
        ).astype(np.float32)
        self.__dataset = ds

    def differential_reflectivity(self):
        return self.__dataset["differential_reflectivity"]

    def __repr__(self) -> str:
        return self.__dataset.__repr__()

    def _repr_html_(self):
        return self.__dataset._repr_html_()

    def to_xarray(self) -> xr.Dataset:
        return self.__dataset

    def to_array(self) -> xr.DataArray:
        return self.__dataset.to_array()

    def to_pandas(self) -> pd.DataFrame:
        return self.__dataset.to_pandas()


data = Nexgen(f)
data
# data[np.isnan(data)] = np.ma.masked

# ri = RadarInterface(f)
# ri[0:2].values
# x = [ray[4] for ray in ri[0][0]]
# x
# np.array(ri[0:2])
# pd.DataFrame(ri[0:2])
# ds = generate(f)

# dict(generate(f))
# pd.DataFrame((generate(f))).set_index([0, 1]).unstack(1)
# [f.sweeps[sweep][0][4][key][1] for key in (b"REF", b"ZDR", b"PHI", b"RHO", b"CFP")]
# np.array([ray[4][b"REF"][1] for ray in f.sweeps[sweep]])
# f.sweeps[0]

<xarray.Dataset>
Dimensions:                    (z: 720, R0: 1832, R1: 1192, Z: 720, i: 721,
                                j: 1832, i1: 721, j1: 1192)
Coordinates:
    ref_range                  (R0) float32 2.125 2.375 2.625 ... 459.6 459.9
    zdr_range                  (R1) float32 2.125 2.375 2.625 ... 299.6 299.9
    sweeps                     (Z) int64 0 1 2 3 4 5 ... 714 715 716 717 718 719
    xloc_0                     (i, j) float64 -2.113 -2.362 ... -457.1 -457.3
    yloc_0                     (i, j) float64 0.2225 0.2486 ... 48.13 48.16
    xloc_1                     (i1, j1) float64 -2.113 -2.362 ... -298.0 -298.2
    yloc_1                     (i1, j1) float64 0.2225 0.2486 ... 31.38 31.4
Dimensions without coordinates: z, R0, R1, Z, i, j, i1, j1
Data variables:
    reflectivity               (z, R0) float32 7.5 9.0 5.5 6.0 ... nan nan nan
    cfp                        (z, R0) float32 35.0 38.0 17.0 ... nan nan nan
    differential_reflectivity  (z, R1) float32 4.844 8.094 10.0 ... nan nan nan
    differential_phase_shift   (z, R1) float32 8.815 282.8 357.9 ... nan nan nan
    correlation_coefficient    (z, R1) float32 0.9717 0.8417 0.825 ... nan nan

In [75]:
from datetime import datetime
import xarray as xr

# define data with variable attributes
data_vars = {
    "velocity": (
        ["x", "y"],
        [[9.8, 19.6, 29.4, 39.2], [9.8, 19.6, 29.4, 39.2], [9.8, 19.6, 29.4, 39.2], [9.8, 19.6, 29.4, 39.2]],
    )
}

# define coordinates
coords = {"time": (["time"], [1, 2, 3, 4]), "x": (["x"], [1, 2, 3, 4])}

# define global attributes
attrs = {"creation_date": datetime.now(), "author": "Jane Doe", "email": "address@email.com"}

# create dataset
ds = xr.Dataset(data_vars=data_vars, coords=coords, attrs=attrs)
ds

<xarray.Dataset>
Dimensions:   (x: 4, y: 4, time: 4)
Coordinates:
  * time      (time) int64 1 2 3 4
  * x         (x) int64 1 2 3 4
Dimensions without coordinates: y
Data variables:
    velocity  (x, y) float64 9.8 19.6 29.4 39.2 9.8 ... 39.2 9.8 19.6 29.4 39.2
Attributes:
    creation_date:  2022-07-27 23:01:46.117324
    author:         Jane Doe
    email:          address@email.com

In [ ]:
# {'ResponseMetadata': {'RequestId': 'CRPSR6BVDSBQXMKF',
#   'HostId': 'pELJx5FV8vxt0Sr5UeRcvx2cGD3/COHa9Z3Js5ajNdSRpiXHLA4+MJ0ZKMtq/iIdNaqhub/RNAE=',
#   'HTTPStatusCode': 200,
#   'HTTPHeaders': {'x-amz-id-2': 'pELJx5FV8vxt0Sr5UeRcvx2cGD3/COHa9Z3Js5ajNdSRpiXHLA4+MJ0ZKMtq/iIdNaqhub/RNAE=',
#    'x-amz-request-id': 'CRPSR6BVDSBQXMKF',
#    'date': 'Wed, 27 Jul 2022 21:55:11 GMT',
#    'last-modified': 'Wed, 20 Jul 2022 00:08:33 GMT',
#    'etag': '"41f7cc0fe551bd6e5e1df90450ce6a0c"',
#    'accept-ranges': 'bytes',
#    'content-type': 'binary/octet-stream',
#    'server': 'AmazonS3',
#    'content-length': '7032059'},
#   'RetryAttempts': 0},
#  'AcceptRanges': 'bytes',
#  'LastModified': datetime.datetime(2022, 7, 20, 0, 8, 33, tzinfo=tzutc()),
#  'ContentLength': 7032059,
#  'ETag': '"41f7cc0fe551bd6e5e1df90450ce6a0c"',
#  'ContentType': 'binary/octet-stream',
#  'Metadata': {},
#  'Body': <botocore.response.StreamingBody at 0x7f8fd8308eb0>}